# **1. Shallow Learning**

In [1]:
import numpy as np
import pickle
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import numpy as np
import pandas as pd
import torch
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from gensim.models import Word2Vec
from tensorflow.keras.utils import to_categorical

In [ ]:
# Cargamos el  dataset tokenizado
df_train = pd.read_pickle("data/data_clean/train_tokenized.pkl")

# Revisamos la columna de etiquetas
print("Clases en 'bias':", df_train["bias"].value_counts())

# Creamos la  carpeta para modelos si no existe
os.makedirs("data/models", exist_ok=True)

# Calculamos TF-IDF sobre df_train 
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2)
)
X_tfidf = tfidf_vectorizer.fit_transform(df_train["tfidf_joined"])
y = df_train["bias"].values

# Separamos el  train/validation 
X_tr, X_val, y_tr, y_val = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42
)

#  Definimos los 4 modelos que vamos a utilizar
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM (Linear)": SVC(kernel='linear'),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=200, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
}

# Entrenamos y evaluamos los modelos
results = {}

for name, model in models.items():
    print(f"Entrenando {name}...")
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='macro')
    results[name] = {"Accuracy": acc, "Macro-F1": f1}
    # Guardar modelo
    pickle.dump(model, open(f"data/models/{name.replace(' ', '_').lower()}.pkl", "wb"))

#  Guardamos el  vectorizador
pickle.dump(tfidf_vectorizer, open("data/features/tfidf_vectorizer.pkl", "wb"))

# Resultados
results_df = pd.DataFrame(results).T
print("\nResultados comparativos de Shallow Learning:")
print(results_df)



Clases en 'bias': bias
2    10240
0     9750
1     7988
Name: count, dtype: int64
Entrenando Logistic Regression...
Entrenando SVM (Linear)...


Para evaluar los modelos hemos usado Accuracy y Macro-F1, métricas adecuadas para problemas de clasificación con clases desbalanceadas.
Los cuatro modelos presentan resultados parecidos. Sin embargo, XGBoost ofrece el mejor rendimiento, mostrando que puede capturar patrones complejos del sesgo ideológico mejor que modelos lineales o Random Forest.
Por otro lado, los modelos lineales funcionan razonablemente bien, lo que indica que el sesgo tiene señales lineales claras en los términos más frecuentes.
Elegimos estos cuatro modelos (Logistic Regression, SVM, Random Forest y XGBoost) para cubrir tanto enfoques lineales como basados en árboles, y utilizar el dataset tokenizado con TF-IDF para representar el texto en forma dispersa, adecuada para Shallow Learning.

# **3. Modelos Deep**


En esta parte nos vamos a enfocar en la clasificación del sesgo. Los motivos son los siguientes:

1- Detectar la orientación política de una noticia es la tarea principal del proyecto. Además, esta tarea permite evaluar cómo los modelos y embeddings capturan matices semánticos y patrones discursivos.
2- Una vez concluida la clasificación de sesgo, se puede reutilizar la pipeline para las demás tareas de clasificación (medio y temática).
3- La columna bias presenta un número moderadamente equilibrado de ejemplos por clase, lo que permite ajustar la arquitectura y los hiperparámetros de manera controlada antes de afrontar tareas más complejas.

Las razones de haber elegido las combinaciones de embeddings y arquitecturas de redes neuronales para abordar la clasificación del sesgo ideológicoson las siguientes:

1- Comparación de embeddings no contextuales y contextuales:
    -Los embeddings no contextuales como, Word2Vecy y FastText, permiten capturar relaciones semánticas entre palabras de manera estática.
    -Los embeddings contextuales como, Sentence Transformers y BERT, capturan el significado de las palabras según su contexto en la frase, lo que es clave para detectar matices ideológicos más complejos.

2- Exploración de diferentes estrategias de embeddings:
    -Word2Vec congelado: usar embeddings preentrenados sin actualizar durante el entrenamiento, para evaluar la capacidad de vectores fijos.
    -Word2Vec fine-tune: ajustar los vectores durante el entrenamiento para adaptarlos al corpus específico.
    -Word2Vec “from scratch”: entrenar desde cero sobre el dataset, para capturar patrones propios del corpus.
    -Para los embeddings contextuales, se compara Sentence Transformers preentrenado frente a BERT, con fine-tuning parcial o total según la arquitectura de la red.

3- Elección de arquitecturas de redes neuronales:
    -Redes totalmente conectadas (Dense): adecuadas para embeddings agregados o promedio de vectores de palabras.
    -Redes recurrentes (LSTM/GRU): capturan secuencias y dependencias entre palabras, esenciales para comprender el flujo discursivo en los artículos.
    -CNN para texto: permiten identificar patrones locales de n-gramas que son relevantes en la clasificación de sesgo.

4- Razonamiento general:
    -Combinar diferentes tipos de embeddings y arquitecturas permite evaluar cuál representa mejor la información semántica y estilística para cada tarea.
    -Esta estrategia también permite analizar cómo el fine-tuning de embeddings impacta en la capacidad del modelo de captar señales ideológicas, frente a vectores preentrenados fijos.

In [5]:
# Cargamos el dataset tokenizado
df_train = pd.read_pickle("data/data_clean/train_tokenized.pkl")
y = df_train["bias"].values

# Codificamos los labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Split train/validation
X_tr_text, X_val_text, y_tr, y_val = train_test_split(
    df_train["tokens"], y_cat, test_size=0.2, random_state=42
)

# Cargamos el Word2Vec preentrenado de la anterior entrega
w2v_model = Word2Vec.load("data/embeddings/word2vec.model")
embedding_dim = w2v_model.vector_size

# Creamos el vocabulario e índices
word_index = {word: i+1 for i, word in enumerate(w2v_model.wv.index_to_key)}
vocab_size = len(word_index) + 1  # +1 para padding

# Convertimos los tokens a índices
def tokens_to_indices(tokens, word_index):
    return [word_index[t] for t in tokens if t in word_index]

X_tr_idx = [tokens_to_indices(t, word_index) for t in X_tr_text]
X_val_idx = [tokens_to_indices(t, word_index) for t in X_val_text]

# Aplciamos padding
max_seq_len = 200
X_tr_pad = pad_sequences(X_tr_idx, maxlen=max_seq_len, padding='post')
X_val_pad = pad_sequences(X_val_idx, maxlen=max_seq_len, padding='post')

# Creamos la matriz de embedding 
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_matrix[i] = w2v_model.wv[word]

# Definimos y entrenaos los modelos 

def build_rnn(model_type='LSTM'):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=max_seq_len,
                        trainable=True))  # Fine-tune embeddings
    if model_type == 'LSTM':
        model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    elif model_type == 'GRU':
        model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# LSTM
lstm_model = build_rnn('LSTM')
lstm_history = lstm_model.fit(X_tr_pad, y_tr,
                              validation_data=(X_val_pad, y_val),
                              epochs=10,
                              batch_size=64)

# GRU
gru_model = build_rnn('GRU')
gru_history = gru_model.fit(X_tr_pad, y_tr,
                            validation_data=(X_val_pad, y_val),
                            epochs=10,
                            batch_size=64)

# Evaluamos los modelos
from sklearn.metrics import accuracy_score, f1_score

# Predicciones
y_pred_lstm = lstm_model.predict(X_val_pad, batch_size=64)
y_pred_gru = gru_model.predict(X_val_pad, batch_size=64)

y_pred_lstm_labels = np.argmax(y_pred_lstm, axis=1)
y_pred_gru_labels = np.argmax(y_pred_gru, axis=1)
y_val_labels = np.argmax(y_val, axis=1)

# Métricas
results = {
    'LSTM': {
        'Accuracy': accuracy_score(y_val_labels, y_pred_lstm_labels),
        'Macro-F1': f1_score(y_val_labels, y_pred_lstm_labels, average='macro')
    },
    'GRU': {
        'Accuracy': accuracy_score(y_val_labels, y_pred_gru_labels),
        'Macro-F1': f1_score(y_val_labels, y_pred_gru_labels, average='macro')
    }
}

results_df = pd.DataFrame(results).T
print("Resultados comparativos de Deep Learning:")
print(results_df)


c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 259s 708ms/step - accuracy: 0.4235 - loss: 1.0572 - val_accuracy: 0.4870 - val_loss: 0.9976
Epoch 2/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 230s 658ms/step - accuracy: 0.5253 - loss: 0.9565 - val_accuracy: 0.4982 - val_loss: 0.9742
Epoch 3/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 206s 589ms/step - accuracy: 0.5179 - loss: 0.9679 - val_accuracy: 0.4089 - val_loss: 1.0904
Epoch 4/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 227s 648ms/step - accuracy: 0.5913 - loss: 0.8691 - val_accuracy: 0.5213 - val_loss: 0.9993
Epoch 5/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 262s 747ms/step - accuracy: 0.6951 - loss: 0.7005 - val_accuracy: 0.5259 - val_loss: 0.9890
Epoch 6/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 241s 689ms/step - accuracy: 0.7779 - loss: 0.5381 - val_accuracy: 0.5381 - val_loss: 1.0824
Epoch 7/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 243s 694ms/step - accuracy: 0.8482 - loss: 0.3815 - val_accuracy: 0.5406 - val_loss: 1.2616
Epoch 8/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 267s 708ms/step - accuracy: 0.9052 -

-Rendimiento general:
   -Ambos modelos muestran resultados muy similares, con valores en torno al 53–54%, tanto en Accuracy como en Macro-F1. Esto indica que:
       -Los dos modelos capturan de forma parecida los patrones secuenciales del sesgo ideológico.
       -No existe una ventaja clara de ninguno de los dos modelos neuronales en este dataset.
-Interpretación:
    -El rendimiento indica que el sesgo ideológico es una tarea difícil incluso para modelos neuronales. 
    -Puede que los textos no tengan suficiente señal secuencial para que LSTM/GRU destaquen claramente.
-Conclusión:
    -Ambos modelos presentan un rendimiento equivalente, pero al ser  ligeramente superior, hemos decidido usar LSTM  como baseline de deep learning. Sin embargo, estas arquitecturas probablemente no capturan matices ideológicos complejos, por lo que se es necesario explorar  modelos más potentes como BERT o RoBERTa.

# **5. Tabla Comaprativa de Resultados**

# **6. Interpretabilidad**